In [6]:
from pycparser.ply.yacc import token
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

# Load dataset
dataset = load_dataset("json", data_files="women_in_tech.jsonl", split="train")

# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples["prompt"] + examples["completion"], truncation=True, padding="max_length", max_length=0)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Fine-tuning arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-women-tech",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

ArrowInvalid: Column 2 named input_ids expected length 20 but got length 40

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset

# Load GPT-2 Medium and Tokenizer
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPT-2 doesn't have a padding token, so we assign one
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# Apply LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,  # Low-rank dimension
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, lora_config)

# Load dataset (example: Hugging Face "wikitext" dataset)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
split = tokenized_dataset.train_test_split(test_size=0.1)  # 10% for evaluation
tokenized_train_dataset = split["train"]
tokenized_eval_dataset = split["test"]




# Training Arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    evaluation_strategy="epoch",  # Prevents empty evaluation loss
    bf16=True  
)


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  # Training dataset
    eval_dataset=tokenized_eval_dataset  # ADD this evaluation dataset
)


# Fine-Tune Model
trainer.train()

# Save Fine-Tuned Model
model.save_pretrained("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")


NameError: name 'train_dataset' is not defined

In [13]:
!which python

/Users/amaljoe/Desktop/Workspace/Research/lm-debugger-proto/.venv/bin/python


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
